In [187]:
"""Importing files and relevant packages and defining global variables"""
import pandas as pd
import numpy as np

data = pd.read_csv('data_cleaned.csv')
SEGMENT_COLS = ['Are you looking for a job?', 'Q13-C22 - Do you work currently? - Answer', 'Have you ever worked before?',
                'D1# City size', 'D2# Gender', 'D3# Age', 'D4# Education']

In [173]:
len(data)

3692

In [47]:
"""Response analysis"""
monthly_net_salary_cols = [col for col in data.columns.tolist() if 'monthly_net_salary' in col and 'expected_' not in col]
exp_monthly_net_salary_cols = [col for col in data.columns.tolist() if 'expected_' in col]

data_slice = data[['id'] + monthly_net_salary_cols + exp_monthly_net_salary_cols + SEGMENT_COLS]
data_slice['monthly_income_num_responses'] = len(monthly_net_salary_cols) - data_slice[monthly_net_salary_cols].isnull().sum(axis=1) 
data_slice['exp_monthly_income_num_responses'] = len(exp_monthly_net_salary_cols) - data_slice[exp_monthly_net_salary_cols].isnull().sum(axis=1) 

C:\Users\tarun.trivedi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\tarun.trivedi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [48]:
data_slice['monthly_income_num_responses'].value_counts(dropna=False)

1    2931
0     682
2      59
3      16
4       2
8       1
6       1
Name: monthly_income_num_responses, dtype: int64

In [49]:
data_slice['exp_monthly_income_num_responses'].value_counts(dropna=False)

1    2195
0     755
2     462
3     123
4      52
5      41
6      34
7      27
8       3
Name: exp_monthly_income_num_responses, dtype: int64

In [50]:
"""Excluding responders who have not provided any response to either monthly income questions 
or exp monthly income questions"""

data_slice = data_slice[(data_slice['monthly_income_num_responses'] > 0) & (data_slice['exp_monthly_income_num_responses'] > 0)]
len(data_slice)

2882

In [51]:
"""Filtering responders who provided info about their current job"""

data_slice['current_job_flag'] = (data_slice.values == 'current job').any(1).astype(int)
data_slice['current_job_flag'].value_counts()

0    1824
1    1058
Name: current_job_flag, dtype: int64

In [53]:
data_slice = data_slice[data_slice['current_job_flag'] == 1]

In [55]:
"""Converting fields current job and sought-after job in data to numeric for ease of analysis"""

data_slice.replace('current job', 1, inplace=True)
data_slice.replace('sought-after job', 1, inplace=True)

In [62]:
"""Cross tab summary"""

def doCrossTabSummary(df):
    summary_clubbed = pd.DataFrame()
    for col in monthly_net_salary_cols:
        df_slice = df[df[col] == 1]
        summary = pd.DataFrame(df_slice[exp_monthly_net_salary_cols].sum())
        summary = summary.transpose()
        summary['income_segment'] = col
        summary = summary[['income_segment'] + exp_monthly_net_salary_cols]
        summary_clubbed = summary_clubbed.append(summary)
    return summary_clubbed


In [70]:
cross_tab_summary = doCrossTabSummary(data_slice)
cross_tab_summary.to_excel('Part-1 Cross Tab Summary.xlsx')
cross_tab_summary

,income_segment,expected_monthly_net_salary_upto_1000eur,expected_monthly_net_salary_1001_1500eur,expected_monthly_net_salary_2001_3000eur,expected_monthly_net_salary_3001_4000eur,expected_monthly_net_salary_4001_5000eur,expected_monthly_net_salary_5001_6000eur,expected_monthly_net_salary_mt_6000eur,expected_monthly_net_salary_refuse_to_answer
0,monthly_net_salary_upto_1000eur,25.0,189.0,203.0,48.0,17.0,12.0,11.0,9.0
0,monthly_net_salary_1001_1500eur,7.0,45.0,226.0,74.0,22.0,20.0,11.0,8.0
0,monthly_net_salary_2001_3000eur,2.0,2.0,66.0,124.0,51.0,28.0,10.0,5.0
0,monthly_net_salary_3001_4000eur,0.0,3.0,6.0,15.0,27.0,18.0,3.0,0.0
0,monthly_net_salary_4001_5000eur,0.0,1.0,2.0,7.0,8.0,20.0,0.0,1.0
0,monthly_net_salary_5001_6000eur,0.0,0.0,3.0,2.0,4.0,4.0,3.0,1.0
0,monthly_net_salary_mt_6000eur,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
0,monthly_net_salary_refuse_to_answer,3.0,12.0,13.0,14.0,11.0,5.0,5.0,42.0


In [245]:
"""Part-2 solution"""

distance_cols = [col for col in data.columns.tolist() if 'distance_from_work' in col]
possible_commute_distance_cols = [col for col in data.columns.tolist() if 'commute_distance' in col]
data_slice_2 = data[['id'] + distance_cols + possible_commute_distance_cols + SEGMENT_COLS + ['Are you keen on moving closer to your work place?']]
data_slice_2['distance_num_responses'] = len(distance_cols) - data_slice_2[distance_cols].isnull().sum(axis=1)
data_slice_2['possible_commute_num_responses'] = len(possible_commute_distance_cols) - data_slice_2[possible_commute_distance_cols].isnull().sum(axis=1)

C:\Users\tarun.trivedi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\tarun.trivedi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [246]:
data_slice_2['Are you keen on moving closer to your work place?'].value_counts(dropna=False)

no     2280
yes     932
NaN     480
Name: Are you keen on moving closer to your work place?, dtype: int64

In [247]:
data_slice_2 = data_slice_2[~pd.isnull(data_slice_2['Are you keen on moving closer to your work place?'])]
len(data_slice_2)

3212

In [248]:
data_slice_2['possible_commute_num_responses'].value_counts(dropna=False)

1    2152
2     382
0     309
3     226
4     113
6      16
5      14
Name: possible_commute_num_responses, dtype: int64

In [249]:
data_slice_2['distance_num_responses'].value_counts(dropna=False)

1    2927
0     200
2      68
3      10
6       4
4       3
Name: distance_num_responses, dtype: int64

In [288]:
tree_data = data_slice_2[(data_slice_2['possible_commute_num_responses'] > 0) & (data_slice_2['distance_num_responses'] > 0)]
len(data_slice_2)

3212

In [289]:
data_slice_2.columns
tree_data = tree_data[['id', 'Are you keen on moving closer to your work place?', 'D4# Education', 'D3# Age', 'D2# Gender', 'D1# City size'] + distance_cols]
tree_data.columns

Index(['id', 'Are you keen on moving closer to your work place?',
       'D4# Education', 'D3# Age', 'D2# Gender', 'D1# City size',
       'distance_from_work_lt_5km', 'distance_from_work_5_10km',
       'distance_from_work_11_20km', 'distance_from_work_21_50km',
       'distance_from_work_51_100km', 'distance_from_work_mt_100km'],
      dtype='object')

In [290]:
"""Transforming numeric variables into categorical for running decision tree"""

features = distance_cols + ['D4# Education', 'D3# Age', 'D2# Gender', 'D1# City size']
tree_data.replace('current job', 1, inplace=True)
tree_data.replace('previous job', 1, inplace=True)
tree_data.replace('sought-after job', 1, inplace=True)
tree_data.replace(np.nan, 0, inplace=True)
tree_data['Are you keen on moving closer to your work place?'] = np.where(tree_data['Are you keen on moving closer to your work place?'] == 'no', 0, 1)
X = pd.get_dummies(tree_data[features], drop_first=True)
y = tree_data['Are you keen on moving closer to your work place?']
y.value_counts(dropna=False)
#X.drop(['commute_distance_mt_100km', 'commute_distance_51_100km'], axis=1, inplace=True)
X.columns

Index(['distance_from_work_lt_5km', 'distance_from_work_5_10km',
       'distance_from_work_11_20km', 'distance_from_work_21_50km',
       'distance_from_work_51_100km', 'distance_from_work_mt_100km',
       'D4# Education_higher', 'D4# Education_secondary',
       'D4# Education_vocational', 'D3# Age_25 - 30 years',
       'D3# Age_31 - 40 years', 'D3# Age_41 - 50 years',
       'D3# Age_less than 18 years', 'D3# Age_more than 50 years',
       'D2# Gender_male', 'D1# City size_200k - 500k inhabitants',
       'D1# City size_20k - 49k inhabitants',
       'D1# City size_50k - 99k inhabitants',
       'D1# City size_city up to 20k inhabitants',
       'D1# City size_more than 500k inhabitants', 'D1# City size_village'],
      dtype='object')

In [291]:
clf = DecisionTreeClassifier(min_samples_leaf = 30)
clf.fit(X, y)
dotfile = open('dtree.dot', 'w')
dot_data = tree.export_graphviz(clf, out_file=dotfile,
                                feature_names=X.columns.tolist())
dotfile.close()

In [261]:
clubbed_summary = pd.DataFrame()
for col in possible_commute_distance_cols:
    verify = tree_data[tree_data[col] == 1]
    verify = pd.DataFrame(verify['Are you keen on moving closer to your work place?'].value_counts(dropna=False)).transpose().reset_index(drop=True)
    verify['possible_commute_distance'] = col
    clubbed_summary = clubbed_summary.append(verify)

clubbed_summary

,0,1,possible_commute_distance
0,884,337,commute_distance_lt_5km
0,951,337,commute_distance_5_10km
0,671,304,commute_distance_11_20km
0,274,207,commute_distance_21_50km
0,24,59,commute_distance_51_100km
0,20,68,commute_distance_mt_100km


In [287]:
dist_51_100 = tree_data[(tree_data['commute_distance_21_50km'] == 1) & (tree_data['Are you keen on moving closer to your work place?'] == 1)]
distri = dist_51_100[distance_cols].sum()
distri

distance_from_work_lt_5km      55.0
distance_from_work_5_10km      42.0
distance_from_work_11_20km     48.0
distance_from_work_21_50km     55.0
distance_from_work_51_100km    12.0
distance_from_work_mt_100km    13.0
dtype: float64

In [295]:
gender = tree_data.groupby(['D2# Gender', 'Are you keen on moving closer to your work place?']).agg({'id':np.size}).reset_index()
gender = gender.transpose()
gender

,0,1,2,3
D2# Gender,female,female,male,male
Are you keen on moving closer to your work place?,0,1,0,1
id,1098,347,924,481
